# script 1 using LinkedIn API

In [1]:
pip install linkedin_api

  Using cached linkedin_api-2.1.1-py3-none-any.whl.metadata (10 kB)
Using cached linkedin_api-2.1.1-py3-none-any.whl (23 kB)



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from linkedin_api import Linkedin
import pandas as pd
import time
import getpass

In [8]:
def login():
    # Setting maximum try limit to 5 log-in attempts
    max_attempts = 5
    attempt = 0
    api = None

    while attempt < max_attempts:
        try:
            user = input('LinkedIN ID: ')

            # To protect User privacy
            passwd = getpass.getpass("Enter your password: ")

            # Logging in in API
            api = Linkedin(user, passwd)
            return api
        except Exception:
            print(f"Login failed. Error: Kindly check your credentials")
            attempt += 1
            if attempt < max_attempts:
                print(f"Remaining attempts: {max_attempts - attempt}")
            else:
                print("Maximum login attempts reached. Exiting.")
                return None


def search_name():
    # Getting name
    name = input('Enter Name : ')
    if name == '':
        print('No name provided! Search will be conducted as per LinkedIn Blank search based on connections!\n')
        return None,None
    try:
        #If full name is provided
        fname,lname = name.split(' ')
        return fname,lname
    except ValueError:
        # If only firstname is provided
        print('Proceeding without a lastname')
        fname = name
        return fname,None


def profile_information_of_top_10_profiles(fname, lname=None):
    print(f"Searching for profiles with first name: {fname} and last name: {lname}")
    
    # Getting URN ID from search_people function of API
    search_results = [urn['urn_id'] for urn in api.search_people(keyword_first_name=fname, keyword_last_name=lname, limit=100)[:10]]
    
    print(f"Search results: {search_results}")
    
    if not search_results:
        print("No search results found.")
        return []
    
    # Getting Profile Information from URN_ID from get_profile function
    profile_info = [pd.DataFrame([api.get_profile(urn_id=id)]) for id in search_results]
    return profile_info


def save_data(profiles):
    try:
        # Getting dataframe with all top entires
        data = pd.concat(profiles,axis=0)

         # Replacing empty lists by NaN
        data = data.applymap(lambda x: None if isinstance(x, list) and len(x) == 0 else x)

        # Getting import information in the first indexes for better readability
        desired_column_order = ['profile_id','firstName','lastName','summary','education','experience','headline','industryName']
        desired_column_order.extend(col for col in data.columns if col not in desired_column_order)
        data = data[desired_column_order]
        
        # Saving data
        data.to_csv('Results')
        print('Data Saved!!!')
        return data  #Just for review purpose
        
    except Exception:
        print('No user found!\nNo data Saved!')
        return None
    

In [12]:
start_time = time.time()

api = login()
if api:
    print('Congratulations, you are permitted to use LinkedIN API\n\n\n')
    first_name,last_name = search_name()
    profiles_info = profile_information_of_top_10_profiles(fname=first_name,lname=last_name)
    data = save_data(profiles_info)

    end_time = time.time()
    total_runtime = end_time - start_time
    print(f"Total runtime: {total_runtime:.2f} seconds")
    
else:
    exit()

Congratulations, you are permitted to use LinkedIN API



Proceeding without a lastname
Searching for profiles with first name: daniel and last name: None
Search results: []
No search results found.
No user found!
No data Saved!
Total runtime: 64.81 seconds
